# Messaging and Requests

## Install

In [ ]:
SYFT_VERSION = ">=0.8.1,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)

In [ ]:
node = sy.orchestra.launch(name="messages-requests-example-domain-1-ds",port=7023, reset=True)

## Setup

For the purpose of this tutorial we are creating a very simple dataset, which is created and owner by the root client

In [ ]:
admin_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
import numpy as np
dataset = sy.Dataset(
    name="my dataset",
    asset_list=[
        sy.Asset(
        name="my asset",
        data=np.array([1,2,3]),
        mock=np.array([1,1,1])
    )]
)
admin_client.upload_dataset(dataset)

In [ ]:
admin_client.register(name="Jane Doe", email="jane@caltech.edu",
                            password="abc123", institution="Caltech", website="https://www.caltech.edu/")

In [ ]:
guest_client = node.client.login(email="jane@caltech.edu", password="abc123")

In [ ]:
asset = guest_client.datasets[0].assets[0]

In [ ]:
@sy.syft_function_single_use()
def func():
    import random
    data = list(range(100))
    return sum(data) / 100 + random.random()

@sy.syft_function_single_use(data=asset)
def mean(data):
    import random
    return sum(data)/len(data) + + random.random()

@sy.syft_function_single_use(data=asset)
def reveal_data(data):
    return data

In [ ]:
# Create a project
new_project = sy.Project(
    name="My Cool Project",
    description="""Hi, I want to calculate the mean of your private data,\
                    pretty please!""",
    members=[guest_client],
)
new_project

In [ ]:
new_project.create_code_request(func, guest_client)
new_project.create_code_request(mean, guest_client)
new_project.create_code_request(reveal_data, guest_client)

In [ ]:
project = new_project.start()
project

In [ ]:
dangerous_request = admin_client.projects[0].requests[-1]
dangerous_request.deny(reason="Dangerous request, you just want to reveal the data!")

## Messaging

list notifications using client.notifications (messages sent and requests)


In [ ]:
guest_client.notifications

## Common Permission Errors

approve request that you dont have permission for example

In [ ]:
guest_client.projects

In [ ]:
guest_client.projects[0]

In [ ]:
func_request = guest_client.projects[0].requests[0]
func_request

In [ ]:
func_request.approve()

## Requesting Changes

request permission to an object via a pointer

In [ ]:
asset = guest_client.datasets[0].assets[0]
asset

In [ ]:
request = asset.pointer.request(guest_client)
request